# $\color{#dda}{\text{FIC: FLOPs Information Criterion}}$ 
## $\color{#dda}{\text{(Remi's Information Criterion - RIC)}}$

**Autor:** Remi Heredia  
**Fecha:** Septiembre 2025  
**Versión:** 2.0

## $\color{#dda}{\text{1. Motivación}}$

Los criterios de información tradicionales (AIC, BIC, MDL) penalizan la complejidad del modelo mediante el **número de parámetros**. Sin embargo, en deep learning moderno:

- Dos modelos con igual número de parámetros pueden tener **complejidad computacional muy diferente**
- Un modelo con más parámetros puede ser más eficiente (ej: MobileNet vs ResNet)
- La arquitectura (depth, skip connections, etc.) no se refleja en el conteo de parámetros

**Ejemplo ilustrativo:**

| Modelo | Parámetros | FLOPs | AIC/BIC |
|--------|-----------|-------|---------|
| Modelo A | 10M | 1 GFLOP | Penaliza 10M |
| Modelo B | 10M | 5 GFLOPs | Penaliza 10M (igual) |

**Problema:** AIC y BIC asignan la misma penalización a ambos modelos, ignorando que el Modelo B requiere 5× más cómputo.

**Propuesta:** Penalizar la complejidad mediante **FLOPs** en lugar de (o además de) parámetros, capturando así el costo computacional real de ejecutar el modelo.

## $\color{#dda}{\text{2. Definición General del FIC}}$


El **FLOPs Information Criterion (FIC)**, también conocido como **Remi's Information Criterion (RIC)**, es un criterio de selección de modelos que penaliza tanto el desajuste a los datos como la complejidad computacional medida en FLOPs.

## $\color{#dda}{\text{3. Relación con Big O y Complejidad Algorítmica}}$


### $\color{#dda}{\text{3.1 Big O vs FLOPs: Diferencias Fundamentales}}$

| Aspecto | Big O | FLOPs (FIC) |
|---------|-------|-------------|
| **Naturaleza** | Teórica/Asintótica | Práctica/Exacta |
| **Pregunta** | "¿Cómo escala con $n$?" | "¿Cuántas operaciones?" |
| **Output** | $O(n^2)$, $O(n \log n)$ | 2,000,000 FLOPs |
| **Constantes** | Ignoradas | Contadas exactamente |
| **Términos menores** | Ignorados | Incluidos |
| **Uso principal** | Análisis de algoritmos | Optimización práctica |

### $\color{#dda}{\text{3.2 Relación Matemática}}$

Big O proporciona la **forma funcional** de los FLOPs:

$
\text{Big O: } f(n) = O(g(n)) \implies \Phi(f) = c \cdot g(n) + \text{términos menores}
$

**Donde:**
- $\Phi(f)$ = FLOPs exactos
- $c$ = constante que Big O ignora
- $g(n)$ = función de crecimiento (ej: $n^2$, $n^3$)



**Ejemplo concreto:**

```
Multiplicación de matrices (n×n)
├─ Big O: O(n³)
└─ FLOPs: 2n³

Para n=100:
├─ Big O: "escala cúbicamente" 
└─ FLOPs: 2,000,000 operaciones exactas
```

### $\color{#dda}{\text{3.3 Por Qué Big O No Es Suficiente para Selección de Modelos}}$

**Caso 1: Misma complejidad asintótica, diferentes FLOPs**

| Operación | Big O | FLOPs |
|-----------|-------|-------|
| Suma de arrays | $O(n)$ | $n$ |
| Producto punto | $O(n)$ | $2n$ |

Ambos son $O(n)$, pero el producto punto requiere **2× más trabajo**.

**Caso 2: Constantes multiplicativas grandes**

```
Algoritmo A: 1000n² operaciones  → O(n²)
Algoritmo B: n³ operaciones      → O(n³)

Para n=10:
├─ Algoritmo A: 100,000 FLOPs (más costoso!)
└─ Algoritmo B: 1,000 FLOPs

Big O diría: "B es peor (cúbico vs cuadrático)"
FLOPs dice: "A es 100× más costoso para n=10"
```

**Caso 3: Términos de orden menor relevantes**

$
\Phi(f) = 2n^3 + 5n^2 + 100n
$

- Big O: $O(n^3)$ (ignora $5n^2 + 100n$)
- Para $n=10$: 
  - Término dominante: 2,000
  - Términos menores: 500 + 1,000 = 1,500 (43% del total!)

### $\color{#dda}{\text{3.4 FIC Captura lo que Big O Ignora}}$

$
\boxed{
\text{FIC captura:}
\begin{cases}
\text{Constantes multiplicativas} & (2n^3 \text{ vs } 5n^3) \\
\text{Términos de orden menor} & (n^3 + n^2 \text{ vs } n^3) \\
\text{Costo exacto para } n \text{ dado} & (1M \text{ vs } 5M \text{ FLOPs})
\end{cases}
}
$

### $\color{#dda}{\text{3.5 Complementariedad}}$

**Big O y FLOPs son complementarios, no excluyentes:**

- **Big O:** Útil para entender escalabilidad y elegir algoritmos
- **FLOPs (FIC):** Útil para optimizar implementaciones y comparar modelos concretos

**Workflow recomendado:**

```
1. Usar Big O para elegir clase de algoritmo
   (e.g., preferir O(n log n) sobre O(n²) para n grande)
   
2. Usar FLOPs/FIC para elegir entre implementaciones
   (e.g., entre dos redes O(n²), elegir la de menos FLOPs)
```

### $\color{#dda}{\text{3.6 Ejemplo en Deep Learning}}$

**Comparación ResNet vs MobileNet:**

```
Big O (ambos):
├─ Entrada: imagen de H×W
├─ Salida: clasificación de C clases
└─ Complejidad: O(H × W × C)

FLOPs (diferentes):
├─ ResNet-50:    4.1 GFLOPs
└─ MobileNetV2:  0.3 GFLOPs

Ratio: 13.7× diferencia!
```

**Big O dice:** "Ambos escalan igual"  
**FIC dice:** "MobileNet es 13.7× más eficiente"

## $\color{#dda}{\text{4. Fórmula Principal}}$


$$
\boxed{
\text{FIC} = -2\log \mathcal{L}(\hat{\theta}|\mathcal{D}) + \alpha \cdot \log(\Phi(f)) + \beta \cdot k
}
$$

### Variables:

| Variable | Nombre | Descripción | Unidades |
|----------|--------|-------------|----------|
| $\text{FIC}$ | FIC score | Valor del criterio (menor es mejor) | escalar real |
| $\mathcal{L}(\hat{\theta}\|\mathcal{D})$ | Verosimilitud | Probabilidad de observar los datos dado el modelo | $[0, 1]$ |
| $\hat{\theta}$ | Parámetros estimados | Parámetros del modelo optimizados en los datos | vector en $\mathbb{R}^k$ |
| $\mathcal{D}$ | Datos | Conjunto de entrenamiento $\{(x_i, y_i)\}_{i=1}^n$ | - |
| $\Phi(f)$ | FLOPs | Número de operaciones de punto flotante del modelo $f$ | entero positivo |
| $f$ | Modelo | Función que mapea entradas a predicciones | $f: \mathcal{X} \to \mathcal{Y}$ |
| $\alpha$ | Peso computacional | Coeficiente de penalización por FLOPs | escalar real, $\alpha \geq 0$ |
| $\beta$ | Peso paramétrico | Coeficiente de penalización por parámetros | escalar real, $\beta \geq 0$ |
| $k$ | Número de parámetros | Dimensión del espacio de parámetros | entero positivo |
| $n$ | Tamaño de muestra | Número de observaciones en $\mathcal{D}$ | entero positivo |

### Valores Recomendados:

- **$\alpha = 2$**: Por analogía con AIC (penalización estándar)
- **$\beta = 1$**: Penalización moderada por parámetros
- **$\alpha = 0, \beta = 0$**: Recupera el criterio de máxima verosimilitud puro

## $\color{#dda}{\text{5. Componentes del FIC}}$

### $\color{#dda}{\text{5.1 Bondad de Ajuste (Goodness-of-fit)}}$

$$
\mathcal{L}_{\text{term}} = -2\log \mathcal{L}(\hat{\theta}|\mathcal{D})
$$

**Variables específicas:**

| Variable | Descripción |
|----------|-------------|
| $\log \mathcal{L}(\hat{\theta}\|\mathcal{D})$ | Log-verosimilitud del modelo |

**Interpretación:**
- Mide qué tan bien el modelo explica los datos observados
- **Menor valor** → mejor ajuste
- Depende del tipo de problema:

#### Para Regresión:
$$
-2\log \mathcal{L}(\hat{\theta}|\mathcal{D}) = n\log\left(\frac{\text{RSS}}{n}\right) + n\log(2\pi) + n
$$

**Variables:**
| Variable | Descripción |
|----------|-------------|
| $\text{RSS}$ | Residual Sum of Squares = $\sum_{i=1}^n (y_i - \hat{y}_i)^2$ |
| $y_i$ | Valor observado de la respuesta para la observación $i$ |
| $\hat{y}_i$ | Predicción del modelo para la observación $i$ |

#### Para Clasificación (Cross-Entropy):
$$
-2\log \mathcal{L}(\hat{\theta}|\mathcal{D}) = -2\sum_{i=1}^{n} \log P(y_i|x_i, \hat{\theta})
$$

**Variables:**
| Variable | Descripción |
|----------|-------------|
| $P(y_i\|x_i, \hat{\theta})$ | Probabilidad asignada por el modelo a la clase verdadera $y_i$ dado input $x_i$ |
| $x_i$ | Vector de características para la observación $i$ |
| $y_i$ | Etiqueta verdadera para la observación $i$ |

### $\color{#dda}{\text{5.2 Término de Penalización Computacional}}$

$$
\mathcal{C}_{\text{comp}} = \alpha \cdot \log(\Phi(f))
$$

**Variables específicas:**

| Variable | Descripción | Cálculo |
|----------|-------------|---------|
| $\Phi(f)$ | FLOPs totales del modelo | Obtenido con `flop_counter.count_model_flops()` |
| $\log(\Phi(f))$ | Escala logarítmica de FLOPs | Reduce impacto de valores muy grandes |

**Casos específicos:**

#### Para Multiplicación de Matrices:
$$
\Phi_{\text{matmul}}(A, B) = 2mnk
$$

**Variables:**
| Variable | Descripción |
|----------|-------------|
| $A \in \mathbb{R}^{m \times k}$ | Primera matriz |
| $B \in \mathbb{R}^{k \times n}$ | Segunda matriz |
| $m$ | Número de filas de $A$ |
| $k$ | Dimensión compartida (columnas de $A$, filas de $B$) |
| $n$ | Número de columnas de $B$ |

#### Para Convolución 2D:
$$
\Phi_{\text{conv2d}} = 2 \cdot H' \cdot W' \cdot C_{\text{out}} \cdot C_{\text{in}} \cdot K_h \cdot K_w
$$

**Variables:**
| Variable | Descripción |
|----------|-------------|
| $H'$ | Altura del feature map de salida |
| $W'$ | Ancho del feature map de salida |
| $C_{\text{out}}$ | Número de canales de salida (filtros) |
| $C_{\text{in}}$ | Número de canales de entrada |
| $K_h$ | Altura del kernel |
| $K_w$ | Ancho del kernel |

#### Para Red Neuronal Feed-Forward:
$$
\Phi_{\text{NN}} = 2\sum_{l=1}^{L} d_{l-1} \cdot d_l
$$

**Variables:**
| Variable | Descripción |
|----------|-------------|
| $L$ | Número de capas |
| $d_l$ | Dimensión de la capa $l$ |
| $d_0$ | Dimensión de entrada |
| $d_L$ | Dimensión de salida |

### $\color{#dda}{\text{5.3 Término de Penalización Paramétrica}}$

$$
\mathcal{C}_{\text{param}} = \beta \cdot k
$$

**Variables específicas:**

| Variable | Descripción |
|----------|-------------|
| $k$ | Número total de parámetros entrenables en el modelo |
| $\beta$ | Factor de penalización (típicamente $\beta = 1$ o $\beta = \log(n)/2$) |

**Cálculo de $k$:**

#### Para Regresión Lineal:
$$
k = p + 1
$$

**Variables:**
| Variable | Descripción |
|----------|-------------|
| $p$ | Número de predictores |
| $+1$ | Término de intercept |

#### Para Red Neuronal:
$$
k = \sum_{l=1}^{L} (d_{l-1} \cdot d_l + d_l)
$$

**Variables:**
| Variable | Descripción |
|----------|-------------|
| $d_{l-1} \cdot d_l$ | Pesos de la conexión entre capa $l-1$ y $l$ |
| $d_l$ | Biases de la capa $l$ |

## $\color{#dda}{\text{6. Variantes del FIC}}$


### $\color{#dda}{\text{6.1 FIC-Estándar (FIC-S)}}$

$$
\text{FIC-S} = -2\log \mathcal{L}(\hat{\theta}|\mathcal{D}) + 2\log(\Phi(f))
$$

**Variables:**
| Variable | Valor | Descripción |
|----------|-------|-------------|
| $\alpha$ | $2$ | Análogo a AIC |
| $\beta$ | $0$ | Solo penaliza FLOPs |

**Uso:** Cuando la complejidad paramétrica es menos relevante que la computacional.

### $\color{#dda}{\text{6.2 FIC-BIC (FIC con penalización tipo Bayesiana)}}$

$$
\text{FIC-BIC} = -2\log \mathcal{L}(\hat{\theta}|\mathcal{D}) + \log(n) \cdot \log(\Phi(f))
$$

**Variables adicionales:**
| Variable | Descripción |
|----------|-------------|
| $\log(n)$ | Penalización que crece con el tamaño de muestra |

**Uso:** Cuando se dispone de muchos datos y se quiere una penalización más conservadora.

### $\color{#dda}{\text{6.3 FIC-Híbrido (FIC-H)}}$

$$
\text{FIC-H} = -2\log \mathcal{L}(\hat{\theta}|\mathcal{D}) + 2\log(\Phi(f)) + k
$$

**Variables:**
| Variable | Valor |
|----------|-------|
| $\alpha$ | $2$ |
| $\beta$ | $1$ |

**Uso:** Balance entre complejidad computacional y paramétrica (**RECOMENDADO**).

### $\color{#dda}{\text{6.4 FIC-Normalizado (FIC-N)}}$

$$
\text{FIC-N} = -2\log \mathcal{L}(\hat{\theta}|\mathcal{D}) + \alpha \cdot \frac{\log(\Phi(f))}{\log(n)}
$$

**Variables adicionales:**
| Variable | Descripción |
|----------|-------------|
| $\log(n)$ | Factor de normalización por tamaño de muestra |

**Uso:** Para comparar modelos entrenados con diferentes tamaños de muestra.

### $\color{#dda}{\text{6.5 FIC-Relativo (FIC-R)}}$

$$
\text{FIC-R} = -2\log \mathcal{L}(\hat{\theta}|\mathcal{D}) + \gamma \cdot \frac{\Phi(f)}{\text{Accuracy}(f)}
$$

**Variables adicionales:**
| Variable | Descripción |
|----------|-------------|
| $\gamma$ | Coeficiente de trade-off eficiencia-precisión |
| $\text{Accuracy}(f)$ | Precisión del modelo en conjunto de validación, $\in [0, 1]$ |

**Uso:** Cuando se quiere penalizar modelos que usan muchos FLOPs pero logran poca mejora en precisión.

## $\color{#dda}{\text{7. Selección de Modelo}}$


### $\color{#dda}{\text{7.1 Criterio de Decisión}}$

Dado un conjunto de modelos candidatos $\mathcal{F} = \{f_1, f_2, \ldots, f_M\}$:

$$
f^* = \arg\min_{f \in \mathcal{F}} \text{FIC}(f)
$$

**Variables:**
| Variable | Descripción |
|----------|-------------|
| $\mathcal{F}$ | Conjunto de modelos candidatos |
| $M$ | Número de modelos candidatos |
| $f^*$ | Modelo óptimo según FIC |

### $\color{#dda}{\text{7.2 Diferencia de FIC (ΔFIC)}}$

Para comparar dos modelos $f_1$ y $f_2$:

$$
\Delta\text{FIC} = \text{FIC}(f_1) - \text{FIC}(f_2)
$$

**Interpretación:**

| $\Delta\text{FIC}$ | Decisión |
|-------------------|----------|
| $\Delta\text{FIC} < -10$ | Evidencia muy fuerte a favor de $f_1$ |
| $-10 \leq \Delta\text{FIC} < -2$ | Evidencia sustancial a favor de $f_1$ |
| $-2 \leq \Delta\text{FIC} \leq 2$ | Ambos modelos son comparables |
| $2 < \Delta\text{FIC} \leq 10$ | Evidencia sustancial a favor de $f_2$ |
| $\Delta\text{FIC} > 10$ | Evidencia muy fuerte a favor de $f_2$ |

## $\color{#dda}{\text{8. Estimación de Hiperparámetros α y β}}$

### $\color{#dda}{\text{8.1 Mediante Validación Cruzada}}$

$$
(\alpha^*, \beta^*) = \arg\min_{(\alpha, \beta) \in \Lambda} \text{CV-Error}(\alpha, \beta)
$$

**Variables:**
| Variable | Descripción |
|----------|-------------|
| $\Lambda$ | Espacio de búsqueda para hiperparámetros |
| $\text{CV-Error}$ | Error de validación cruzada promedio |

**Procedimiento:**

1. Definir grid de búsqueda:
   $$
   \Lambda = \{(\alpha, \beta) : \alpha \in [0, 5], \beta \in [0, 5]\}
   $$

2. Para cada par $(\alpha, \beta)$:
   - Calcular $\text{FIC}(f; \alpha, \beta)$ para todos los modelos
   - Seleccionar $f^*(\alpha, \beta) = \arg\min_f \text{FIC}(f; \alpha, \beta)$
   - Evaluar error en validación: $e(\alpha, \beta)$

3. Seleccionar:
   $$
   (\alpha^*, \beta^*) = \arg\min_{(\alpha,\beta)} e(\alpha, \beta)
   $$

### $\color{#dda}{\text{8.2 Valores Teóricos por Analogía}}$

#### Analogía con AIC:
$$
\alpha = 2, \quad \beta = 0
$$

#### Analogía con BIC:
$$
\alpha = \frac{\log(n)}{2}, \quad \beta = 0
$$

#### Híbrido (Recomendado):
$$
\alpha = 2, \quad \beta = 1
$$

## $\color{#dda}{\text{9. Propiedades Matemáticas del FIC}}$


### $\color{#dda}{\text{9.1 Consistencia}}$

**Definición:** El FIC es consistente si:

$$
\lim_{n \to \infty} P(\text{FIC selecciona el modelo verdadero } f_0) = 1
$$

**Variables:**
| Variable | Descripción |
|----------|-------------|
| $f_0$ | Modelo verdadero (generador de datos) |
| $P(\cdot)$ | Probabilidad |

**Condiciones suficientes:**
- $\alpha = o(\sqrt{n})$
- $\beta = o(\sqrt{n})$
- El modelo verdadero está en el conjunto de candidatos

### $\color{#dda}{\text{9.2 Invarianza a Escala}}$

Para una transformación de FLOPs $\Phi'(f) = c \cdot \Phi(f)$ con $c > 0$:

$$
\text{FIC}'(f) = -2\log \mathcal{L}(\hat{\theta}|\mathcal{D}) + \alpha \cdot \log(c \cdot \Phi(f)) + \beta \cdot k
$$

$$
= \text{FIC}(f) + \alpha \log(c)
$$

**Variables:**
| Variable | Descripción |
|----------|-------------|
| $c$ | Constante de escala positiva |
| $\alpha \log(c)$ | Desplazamiento constante (no afecta ranking de modelos) |

**Conclusión:** El ranking relativo de modelos se preserva bajo transformaciones de escala.

## $\color{#dda}{\text{10. Casos de Uso Específicos}}$


### $\color{#dda}{\text{10.1 Regresión Lineal Simple}}$

**Modelo:**
$$
y_i = \beta_0 + \beta_1 x_i + \epsilon_i, \quad \epsilon_i \sim \mathcal{N}(0, \sigma^2)
$$

**FIC:**
$$
\text{FIC} = n\log(\text{RSS}) + 2\log(3np) + 2
$$

**Variables:**
| Variable | Descripción | Valor |
|----------|-------------|-------|
| $\text{RSS}$ | Suma de residuos cuadrados | $\sum_{i=1}^n (y_i - \hat{y}_i)^2$ |
| $3np$ | FLOPs para regresión lineal | $2np$ (producto matriz-vector) + $np$ (residuos) |
| $p$ | Número de predictores | En regresión simple, $p=1$ |
| $k$ | Parámetros | $k = 2$ ($\beta_0, \beta_1$) |

### $\color{#dda}{\text{10.2 Red Neuronal Multicapa}}$

**Arquitectura:** $[d_0, d_1, d_2, \ldots, d_L]$

**FIC:**
$$
\text{FIC} = -2\sum_{i=1}^n \log P(y_i|x_i) + 2\log\left(2\sum_{l=1}^L d_{l-1}d_l\right) + \sum_{l=1}^L (d_{l-1}d_l + d_l)
$$

**Variables:**
| Variable | Descripción |
|----------|-------------|
| $d_0$ | Dimensión de entrada (features) |
| $d_l$ | Número de neuronas en capa $l$ |
| $d_L$ | Dimensión de salida (clases o outputs) |
| $L$ | Número de capas (profundidad de la red) |
| $P(y_i\|x_i)$ | Softmax output para clasificación o Gaussiana para regresión |

### $\color{#dda}{\text{10.3 Modelo Convolucional (CNN)}}$

**Para una sola capa convolucional:**

$$
\text{FIC}_{\text{conv}} = -2\log \mathcal{L} + 2\log(2H'W'C_{\text{out}}C_{\text{in}}K_hK_w) + (C_{\text{out}} \cdot C_{\text{in}} \cdot K_h \cdot K_w + C_{\text{out}})
$$

**Variables:**
| Variable | Descripción | Rango típico |
|----------|-------------|--------------|
| $H', W'$ | Dimensiones espaciales de salida | Depende de stride y padding |
| $C_{\text{in}}$ | Canales de entrada | RGB: 3, grayscale: 1 |
| $C_{\text{out}}$ | Número de filtros | 16-512 típicamente |
| $K_h, K_w$ | Tamaño del kernel | 3×3, 5×5, 7×7 común |

**Para arquitectura completa:**
$$
\text{FIC}_{\text{CNN}} = -2\log \mathcal{L} + 2\log\left(\sum_{\text{capas}} \Phi_{\text{capa}}\right) + \sum_{\text{capas}} k_{\text{capa}}
$$

## $\color{#dda}{\text{11. Resumen de Notación Completa}}$

| Símbolo | Nombre | Descripción | Tipo |
|---------|--------|-------------|------|
| $\text{FIC}$ | FIC score | Valor del criterio | escalar |
| $\mathcal{L}$ | Verosimilitud | Probabilidad de los datos | $[0,1]$ |
| $\hat{\theta}$ | Parámetros | Estimaciones MLE/MAP | vector |
| $\mathcal{D}$ | Datos | Conjunto de entrenamiento | conjunto |
| $\Phi(f)$ | FLOPs | Operaciones de punto flotante | entero |
| $f$ | Modelo | Función predictiva | función |
| $\alpha$ | Peso FLOPs | Penalización computacional | real |
| $\beta$ | Peso parámetros | Penalización paramétrica | real |
| $k$ | Parámetros | Dimensión de $\theta$ | entero |
| $n$ | Tamaño muestra | Número de observaciones | entero |
| $p$ | Predictores | Número de features | entero |
| $L$ | Capas | Profundidad de red | entero |
| $d_l$ | Neuronas | Ancho de capa $l$ | entero |
| $\text{RSS}$ | Residuos | Suma de errores cuadrados | real |

## $\color{#dda}{\text{12. Fórmula Final (RECOMENDADA)}}$

$$
\boxed{
\begin{aligned}
\text{FIC} &= -2\log \mathcal{L}(\hat{\theta}|\mathcal{D}) + 2\log(\Phi(f)) + k \\
&= \underbrace{-2\log \mathcal{L}}_{\text{ajuste}} + \underbrace{2\log(\Phi)}_{\text{complejidad computacional}} + \underbrace{k}_{\text{complejidad paramétrica}}
\end{aligned}
}
$$

**Dónde:**
- **Primer término:** Mide qué tan bien el modelo se ajusta a los datos (menor es mejor)
- **Segundo término:** Penaliza modelos computacionalmente costosos (FLOPs altos)
- **Tercer término:** Penaliza modelos con muchos parámetros (previene overfitting)

**Interpretación:** Busca el modelo que mejor balancea ajuste, eficiencia computacional y simplicidad paramétrica.